In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import fitz
import pytesseract
from PIL import Image

In [6]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [7]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

In [14]:
# In case of .pdf file
pages = []
file = '6.pdf'
doc = fitz.open(file)
for n in range(doc.page_count):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    pages.append(image)

In [10]:
image = pages[3]
# Отображение изображения с выделенными ячейками
cv2.imshow('Image with Cells', image)
cv2.imwrite('image.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


Для выделения таблицы из корпуса текста попробуем использовать cv2 с такими настройками

In [11]:
# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек

height, width, _ = image.shape

parts_image = []

for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    x, y, w, h = cv2.boundingRect(contour)

    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

        cropp_image = image[y:y + h, x:x + w]

        parts_image.append(cropp_image)

# Вывод координат ограничивающих рамок ячеек
for i, box in enumerate(cell_boxes):
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow('Image with Cells', parts_image[i])
    cv2.imwrite('parts_image.jpg', parts_image[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# # Отображение изображения с выделенными ячейками
# cv2.imshow('Image with Cells', image)
# cv2.imwrite('image_boxes.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [15]:
# Загрузка изображения
image = parts_image[0].copy()

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, threshold_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((2, 2), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

inv_image = cv2.bitwise_not(morph_image)

# Отображение изображения с ограничивающими рамками символов
cv2.imshow('Image with Character Boxes', inv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [16]:
# Загрузка изображения
image = parts_image[0]

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, threshold_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)

# Получение ограничивающих рамок символов
boxes = pytesseract.image_to_boxes(threshold_image, lang='rus')

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# print(boxes_list)

# Так как cv2 и tesseract используют разные системы координат
height, width = gray_image.shape

# Список границ таблицы
borders = []

epsilon = 10

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~' and ((abs(int(box[1]) - int(box[3])) <= epsilon) or (abs(int(box[2]) - int(box[4])) <= epsilon)):
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        cv2.rectangle(image, (x, height - y), (w, height - h), (255, 0, 255), 1)

# Отображение изображения с ограничивающими рамками символов
cv2.imshow('Image with Character Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


Попробуем найти координаты всех узлов таблицы, по этим узлам восстановим прямоугольники ячеек

In [17]:
import itertools

# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 10

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((int((x1 + x2) / 2), y1, int((x1 + x2) / 2), y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, int((y1 + y2) / 2), x2, int((y1 + y2) / 2)))

# Находим координаты узлов таблицы
table_nodes = []
# for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
#     x, y = v_line[0], h_line[1]
#     print(x, y)
#     cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
#     table_nodes.append((x, y))

for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    v_x1, v_y1, v_x2, v_y2 = v_line
    h_x1, h_y1, h_x2, h_y2 = h_line

    # Проверяем, пересекаются ли прямые или заканчивается ли одна из них в окрестности
    if ((h_x1 - epsilon <= v_x1 <= h_x2 + epsilon or h_x1 - epsilon <= v_x2 <= h_x2 + epsilon) and
        (v_y1 - epsilon <= h_y1 <= v_y2 + epsilon or v_y1 - epsilon <= h_y2 <= v_y2 + epsilon)) or (
            abs(h_x2 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x2 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x1 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x1 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2):
        # Добавляем координаты узла в список
        table_nodes.append((v_x1, h_y1))

    if v_x1 <= epsilon or v_y1 <= epsilon or abs(v_x1 - width) <= epsilon or abs(v_y1 - height) <= epsilon:
        table_nodes.append((v_x1, v_y1))

    if v_x2 <= epsilon or v_y2 <= epsilon or abs(v_x2 - width) <= epsilon or abs(v_y2 - height) <= epsilon:
        table_nodes.append((v_x2, v_y2))

    if h_x1 <= epsilon or h_y1 <= epsilon or abs(h_x1 - width) <= epsilon or abs(h_y1 - height) <= epsilon:
        table_nodes.append((h_x1, h_y1))

    if h_x2 <= epsilon or h_y2 <= epsilon or abs(h_x2 - width) <= epsilon or abs(h_y2 - height) <= epsilon:
        table_nodes.append((h_x2, h_y2))

# Создаем копию списка table_nodes для модификации
modified_table_nodes = table_nodes.copy()

# Проходим по каждой точке в списке table_nodes
for i, node in enumerate(table_nodes):
    x, y = node
    count = 1
    avg_x, avg_y = x, y

    # Ищем точки в окрестности epsilon
    for j, other_node in enumerate(table_nodes):
        if i != j:  # Игнорируем текущую точку
            other_x, other_y = other_node

            # Проверяем, находится ли другая точка в окрестности epsilon
            if abs(other_x - x) <= epsilon and abs(other_y - y) <= epsilon:
                avg_x += other_x
                avg_y += other_y
                count += 1

    # Если найдены точки в окрестности, заменяем текущую точку на среднее значение
    if count > 1:
        avg_x /= count
        avg_y /= count
        modified_table_nodes[i] = (int(avg_x), int(avg_y))

f = True

while f:
    for i, node in enumerate(modified_table_nodes):
        if i == len(modified_table_nodes) - 1:
            f = False
        else:
            x, y = node
            count = 1
            indices = []
            avg_x, avg_y = x, y

            # Ищем точки в окрестности epsilon
            for j, other_node in enumerate(modified_table_nodes):
                if i != j:  # Игнорируем текущую точку
                    other_x, other_y = other_node

                    # Проверяем, находится ли другая точка в окрестности epsilon
                    if abs(other_x - x) <= epsilon and abs(other_y - y) <= epsilon:
                        avg_x += other_x
                        avg_y += other_y
                        count += 1
                        indices.append(j)

            # Если найдены точки в окрестности, заменяем текущую точку на среднее значение
            if count > 1:
                avg_x /= count
                avg_y /= count
                modified_table_nodes[i] = (int(avg_x), int(avg_y))

            for item in sorted(indices, reverse=True):
                modified_table_nodes.pop(item)

# Заменяем исходный список table_nodes на модифицированный
table_nodes = modified_table_nodes

# Выводим координаты узлов на изображении
for node in table_nodes:
    x, y = node
    print(x, y)
    cv2.rectangle(image, (x - 3, height - y - 3), (x + 3, height - y + 3), (0, 0, 255), 1)

# Отображение изображения
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1 273
482 273
482 241
482 188
0 172
482 172
0 140
482 140
305 99
0 99
482 99
0 59
482 59
0 29
482 29
206 7
223 7
0 0
482 0
47 270
47 0
47 172
47 140
47 99
47 59
47 29
146 270
146 0
146 250
146 188
146 172
146 140
146 99
146 59
146 29
276 270
276 0
276 250
276 188
276 172
276 140
276 99
276 59
276 29
323 270
323 0
323 241
323 226
323 188
323 172
323 140
323 99
323 59
323 29
189 0
189 250
189 188
189 172
189 140
189 99
189 59
189 29
234 0
234 250
234 188
234 172
234 140
234 99
234 59
234 29
446 0
446 241
446 226
446 188
446 172
446 140
446 99
446 59
446 29
365 0
365 226
365 188
365 172
365 140
365 99
365 59
365 29
409 0
409 226
409 188
409 172
409 140
409 99
409 59
409 29


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [18]:
# Замена координат, отличающихся менее чем на epsilon
rounded_nodes = np.round(np.array(table_nodes) / epsilon) * epsilon

# Подсчет уникальных значений координат x и y
unique_x = np.unique(rounded_nodes[:, 0])
unique_y = np.unique(rounded_nodes[:, 1])

# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1

print(num_rows, num_columns)

# Отсортируем узлы по оси x
nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], - x[1]))
print(nodes_sorted_x)

# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_x) - 1):
    current_node = nodes_sorted_x[i]
    if nodes_sorted_x[i + 1][0] == current_node[0]:
        next_y_node = nodes_sorted_x[i + 1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_x_nodes = [node for node in nodes_sorted_x if abs(node[1] - current_node[1]) <= epsilon and node[0] > current_node[0]]
    for next_x_node in next_x_nodes:
        opposite_node = (next_x_node[0], next_y_node[1])
        for node in nodes_sorted_x:
            if (node[0] - opposite_node[0]) <=epsilon and (node[1] - opposite_node[1]):
                # Построим прямоугольник по найденным узлам
                rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
                rectangles.append(rectangle)
                cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (255, 255, 0), 1)
            else:
                break
    else:
        continue
# Выведем список прямоугольников
# print(rectangles)

# Отображение изображения
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

11 13
[(0, 172), (0, 140), (0, 99), (0, 59), (0, 29), (0, 0), (1, 273), (47, 270), (47, 172), (47, 140), (47, 99), (47, 59), (47, 29), (47, 0), (146, 270), (146, 250), (146, 188), (146, 172), (146, 140), (146, 99), (146, 59), (146, 29), (146, 0), (189, 250), (189, 188), (189, 172), (189, 140), (189, 99), (189, 59), (189, 29), (189, 0), (206, 7), (223, 7), (234, 250), (234, 188), (234, 172), (234, 140), (234, 99), (234, 59), (234, 29), (234, 0), (276, 270), (276, 250), (276, 188), (276, 172), (276, 140), (276, 99), (276, 59), (276, 29), (276, 0), (305, 99), (323, 270), (323, 241), (323, 226), (323, 188), (323, 172), (323, 140), (323, 99), (323, 59), (323, 29), (323, 0), (365, 226), (365, 188), (365, 172), (365, 140), (365, 99), (365, 59), (365, 29), (365, 0), (409, 226), (409, 188), (409, 172), (409, 140), (409, 99), (409, 59), (409, 29), (409, 0), (446, 241), (446, 226), (446, 188), (446, 172), (446, 140), (446, 99), (446, 59), (446, 29), (446, 0), (482, 273), (482, 241), (482, 188), (

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [310]:
rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
print(rectangles)

[((0, 266), (47, 99)), ((47, 266), (146, 172)), ((146, 266), (280, 250)), ((280, 266), (323, 250)), ((323, 266), (483, 241)), ((146, 250), (189, 188)), ((189, 250), (234, 188)), ((234, 250), (280, 188)), ((323, 241), (446, 226)), ((446, 241), (483, 226)), ((323, 226), (365, 188)), ((365, 226), (409, 188)), ((409, 226), (446, 188)), ((146, 188), (189, 172)), ((189, 188), (234, 172)), ((234, 188), (276, 172)), ((276, 188), (323, 172)), ((323, 188), (365, 172)), ((365, 188), (409, 172)), ((409, 188), (446, 172)), ((446, 188), (483, 172)), ((1, 172), (47, 140)), ((47, 172), (146, 140)), ((146, 172), (189, 140)), ((189, 172), (234, 140)), ((234, 172), (276, 140)), ((276, 172), (323, 140)), ((323, 172), (365, 140)), ((365, 172), (409, 140)), ((409, 172), (446, 140)), ((446, 172), (482, 140)), ((1, 140), (47, 29)), ((47, 140), (146, 99)), ((146, 140), (189, 99)), ((189, 140), (234, 99)), ((234, 140), (276, 99)), ((276, 140), (323, 99)), ((323, 140), (365, 99)), ((365, 140), (409, 99)), ((409,

In [313]:
# Создание пустого DataFrame для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[max(0, (height - y2) - 2):min(height, (height - y1) + 2),
                 max(0, x1 - 2):min(width, x2 + 2)]

    # Russian
    custom_config = r'--oem 1 --psm 1 -l rus'

    if cell_image.shape[0] > 10 and cell_image.shape[1] > 10:
        gray_image = cv2.cvtColor(cell_image, cv2.COLOR_BGR2GRAY)
        _, threshold_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
        # Отображение изображения
        cv2.imshow('Image', threshold_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Распознавание текста с помощью Tesseract
        text = pytesseract.image_to_string(threshold_image, config=custom_config).replace('\n', ' ')

        print(text)

        # Определение положения ячейки в таблице
        row = i // num_columns
        column = i % num_columns

        # Добавление данных ячейки
        if row < num_rows and column < num_columns:
            table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)